In [1]:
"""
Jason Stranne
"""
import numpy as np
import os
import sys
import gc
from All_Three_Combined_Downstream import DownstreamNet_Combined, Downstream_Dataset, print_class_counts, num_correct, reduce_dataset_size
from All_Three_Combined_Downstream import smallest_class_len, restrict_training_size_per_class, train_end_to_end
sys.path.insert(0, '..')
from Stager_net_pratice import StagerNet
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import LeaveOneGroupOut, GroupKFold
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset


In [2]:
def train_different_classes(model_patha, model_pathb, model_pathc, tset, vset, epochs, sample_list):
    outList=[]
    balanced_acc_out=[]
    for i in sample_list:
        acc, balanced_acc = train_end_to_end(model_patha, model_pathb, model_pathc, tset, vset, i, epochs, 3)
        outList.append(acc)
        balanced_acc_out.append(balanced_acc)
    return outList, balanced_acc_out

# Models

In [5]:
RP_model_path = "RP_stagernet_30_120.pth"
TS_model_path = "TS_stagernet_30_30.pth"
CPC_model_path = "CPC_stagernet_16.pth"

In [6]:
root = os.path.join("Mouse_Training_Data", "Windowed_Data", "")
datasets_list=[]
print('Loading Data')
f=open(os.path.join("training_names.txt"),'r')
lines = f.readlines()
x_vals = []
y_vals = []
groups = []
index = 0
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    datasets_list.append(Downstream_Dataset(path=data_file))
    d = Downstream_Dataset(path=data_file)
    x_vals.append(d.data)
    y_vals.append(d.labels)
    groups.append(np.ones(len(d.labels))*index)
    index+=1
f.close()


x_vals = np.vstack(x_vals)
y_vals = np.concatenate(y_vals, axis=0)
groups = np.concatenate(groups, axis=0)
print(x_vals.shape)
print(y_vals.shape)
print(groups.shape)


# logo = LeaveOneGroupOut()
kfold = GroupKFold(n_splits=5)
# logo.get_n_splits(x_vals, y_vals, groups)
kfold.get_n_splits(x_vals, y_vals, groups)
                    
                    
result_dict = {}
# dtype=torch.int32
for train_index, test_index in kfold.split(x_vals, y_vals, groups):
    unique = np.unique(groups[test_index])
    # group_num = groups[test_index][0]
    print("Leaving out mouse number:", unique)
    training_set = TensorDataset(torch.tensor(x_vals[train_index], dtype=torch.float), torch.tensor(y_vals[train_index], dtype=torch.long))
    test_set = TensorDataset(torch.tensor(x_vals[test_index], dtype=torch.float), torch.tensor(y_vals[test_index], dtype=torch.long))
    print(len(training_set))
    smallest_class = smallest_class_len(training_set, 3)
    num_samples=[]
    temp=1
    while temp < smallest_class:
        num_samples.append(temp)
        temp*=10
    num_samples.append(None)
    accuracy, balanced_accuracy = train_different_classes(
        RP_model_path,
        TS_model_path,
        CPC_model_path,
        training_set, 
        test_set, 
        100, 
        num_samples
    )
    for num_pos, acc in zip(num_samples, accuracy):
        if num_pos not in result_dict:
            result_dict[num_pos] = []
        result_dict[num_pos].append(acc)
    print(result_dict)
    
for k in result_dict:
    result_dict[k] = np.mean(result_dict[k])
print(result_dict)



Loading Data
Processing MouseCKA1_030515_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKB9_022715_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKL1_062514_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKL5_063014_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKL7_063014_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKN1_063014_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)


In [7]:
{1: 0.54925, 10: 0.8557499999999999, 100: 0.8901, 1000: 0.9251166666666666, None: 0.9473499999999999}
for first trial
{1: 0.6627, 10: 0.8325333333333333, 100: 0.8905166666666666, 1000: 0.93125, None: 0.9473499999999999}
second
{1: 0.6797000000000001, 10: 0.8249166666666667, 100: 0.89275, 1000: 0.9317333333333334, None: 0.9477666666666666}

SyntaxError: invalid syntax (<ipython-input-7-4f2482a33e6f>, line 2)

In [8]:
trial1=[0.54925, 0.8557499999999999, 0.8901, 0.9251166666666666, 0.9473499999999999]
trial2=[0.6627, 0.8325333333333333, 0.8905166666666666, 0.93125, 0.9473499999999999]
trial3=[0.6797000000000001, 0.8249166666666667, 0.89275, 0.9317333333333334, 0.9477666666666666]
avg=[]
for i in range(len(trial1)):
    avg.append((trial1[i]+trial2[i]+trial3[i])/3)
print(avg)

[0.6305499999999999, 0.8377333333333333, 0.8911222222222221, 0.9293666666666667, 0.9474888888888889]


In [4]:
smallest_class = smallest_class_len(training_set)
num_samples=[]
temp=1
while temp < smallest_class:
    num_samples.append(temp)
    temp*=10
num_samples.append(None)
print(num_samples)

[1, 10, 100, 1000, None]


{1: 0.6547992424242424, 10: 0.8585719696969697, 100: 0.9048257575757577, 1000: 0.938746212121212, None: 0.9544659090909091}


[0.6547992424242424, 0.8585719696969697, 0.9048257575757577, 0.938746212121212, 0.9544659090909091]

In [9]:
CPC_vals = train_different_classes("RP_stagernet.pth", "TS_stagernet.pth", "CPC_stagernet.pth", training_set, validation_set, 100, num_samples)

len of the dataloader is: 1
Start Training
[0.33246640658864324]
len of the dataloader is: 1
Start Training
[0.33246640658864324, 0.4729085392284352]
len of the dataloader is: 2
Start Training
[0.33246640658864324, 0.4729085392284352, 0.5831816211530125]
len of the dataloader is: 20
Start Training
[0.33246640658864324, 0.4729085392284352, 0.5831816211530125, 0.6476809709579541]
len of the dataloader is: 68
Start Training
[0.33246640658864324, 0.4729085392284352, 0.5831816211530125, 0.6476809709579541, 0.7046380580840919]


In [6]:
print(CPC_vals)

[0.28097095795405286, 0.42418725617685304, 0.5448634590377113, 0.5814477676636324, 0.6376246207195492]
